In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from dotenv import load_dotenv

In [2]:
import os
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['SERPAPI_API_KEY'] = os.getenv("SERPAPI_API_KEY")


In [5]:
%pip install google-search-results

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install numexpr

In [12]:
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
# tools = ["serpapi", "llm-math"]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION , verbose = True )


In [13]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")




> Entering new AgentExecutor chain...
 I should use the search engine to find information about Leo DiCaprio's girlfriend.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Vittoria Ceretti
Thought: Now that I know her name, I can use the search engine again to find her age.
Action: Search
Action Input: "Vittoria Ceretti age"
Observation: 26 years
Thought: I can use the calculator to raise her age to the 0.43 power.
Action: Calculator
Action Input: 26, 0.43
Observation: Answer: 26.43
Thought: I now know the final answer.
Final Answer: Vittoria Ceretti's current age raised to the 0.43 power is 26.43.

> Finished chain.


"Vittoria Ceretti's current age raised to the 0.43 power is 26.43."

## SQL DATABASE INTEGRATION

In [15]:
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor

In [19]:
%pip install pymysql

  Using cached PyMySQL-1.1.1-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.1-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
db_user = "####"
db_password = "####"
db_host = "localhost"  
db_port = "3306"  
db_name = "classicmodels"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

In [32]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
# llm = OpenAI(temperature=0)

In [33]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(llm, toolkit, verbose=True)


In [34]:
agent_executor.run("Describe the Order related table and how they are related")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI should query the schema of the tables related to orders to understand their structure and relationships.
Action: sql_db_schema
Action Input: orders, orderdetails
CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from orderdetails table:
orderNumber	productCode	quantityOrdered	priceEach	orderLineNumber
10100	S18_1749	30	136.00	3
10100	S18_2248	50	

'The Order related tables are "orders" and "orderdetails". The "orderdetails" table has a foreign key constraint referencing the "orders" table on the "orderNumber" column.'

In [35]:
agent_executor.run("Describe the PurchaseDetails table")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI should query the schema of the PurchaseDetails table to describe it.
Action: sql_db_schema
Action Input: PurchaseDetailsError: table_names {'PurchaseDetails'} not found in databaseI should double check the table name to ensure it is correct.
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsThe table name is not PurchaseDetails, I should try to find the correct name by looking at the schema of the tables.
Action: sql_db_schema
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, products
CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`ad

"I don't know"

In [37]:
agent_executor.run("Describe OrderDetails table")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI should query the schema of the OrderDetails table to describe it.
Action: sql_db_schema
Action Input: OrderDetailsError: table_names {'OrderDetails'} not found in databaseI should double check the table name to make sure it is correct.
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsThe table name is correct, so I should try querying the schema of the OrderDetails table again.
Action: sql_db_schema
Action Input: OrderDetailsError: table_names {'OrderDetails'} not found in databaseI should check the spelling of the table name and try querying the schema again.
Action: sql_db_schema
Action Input: orderdetails
CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NUL

'The OrderDetails table contains information about orders, including the product codes, quantities ordered, prices, and order line numbers.'

In [38]:
agent_executor.run("Find the top 5 products with the highest total sales revenue")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI should query the schema of the products and orderdetails tables to find the relevant columns for total sales revenue.
Action: sql_db_schema
Action Input: products, orderdetails
CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from orderdetails table:
orderNumber	productCode	quantityOrdered	priceEach	orderLineNumber
10100	S18_1749	30	136.00	3
101

'The top 5 products with the highest total sales revenue are:\n1. Product Code: S18_3232, Product Name: 1992 Ferrari 360 Spider red, Total Revenue: $276,839.98\n2. Product Code: S12_1108, Product Name: 2001 Ferrari Enzo, Total Revenue: $190,755.86\n3. Product Code: S10_1949, Product Name: 1952 Alpine Renault 1300, Total Revenue: $190,017.96\n4. Product Code: S10_4698, Product Name: 2003 Harley-Davidson Eagle Drag Bike, Total Revenue: $170,686.00\n5. Product Code: S12_1099, Product Name: 1968 Ford Mustang, Total Revenue: $161,531.48'

In [40]:
agent_executor.run("List the top 3 countries with the highest number of orders")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI should query the orders table to find the number of orders per country.
Action: sql_db_schema
Action Input: orders
CREATE TABLE orders (
	`orderNumber` INTEGER NOT NULL, 
	`orderDate` DATE NOT NULL, 
	`requiredDate` DATE NOT NULL, 
	`shippedDate` DATE, 
	status VARCHAR(15) NOT NULL, 
	comments TEXT, 
	`customerNumber` INTEGER NOT NULL, 
	PRIMARY KEY (`orderNumber`), 
	CONSTRAINT orders_ibfk_1 FOREIGN KEY(`customerNumber`) REFERENCES customers (`customerNumber`)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from orders table:
orderNumber	orderDate	requiredDate	shippedDate	status	comments	customerNumber
10100	2003-01-06	2003-01-13	2003-01-10	Shipped	None	363
10101	2003-01-09	2003-01-18	2003-01-11	Shipped	Check on availability.	128
10102	2003-01-10	2003-01-18	2003-01-14	Shipped	None	181
*

'The top 3 countries with the highest number of orders are USA, France, and Spain.'